In [1]:
#In this notebook:
#1. This notebook compares Raw Model with Embedding  of train on absolute gene expression
#2. Creates a stacked ensemble of Raw and Embedding models
%cd "~/pln-brca-xp/notebooks"
%load_ext autoreload
%autoreload 2

/home/xabush/pln-brca-xp/notebooks


In [41]:
from utils import *

In [3]:
ge_df = pd.read_csv("/var/www/datasets/merged-combat15.csv.xz", index_col="patient_ID")
state_df = pd.read_csv("/var/www/datasets/embedding_vector_state_and_outcome.csv", index_col="patient_ID")
ge_out_df = ge_df.join(state_df["posOutcome"])
X_ge_train, X_ge_test, y_ge_train, y_ge_test = get_train_test_set(ge_out_df, "/var/www/datasets/tamox_train_set.txt", "/var/www/datasets/tamox_test_set.txt")
X_ge_train

,MAGEA12,MAGEA11,KLF1,ADH7,MSH4,BIRC3,AKR1C4,GBX2,GCGR,SIGLEC9,...,ZNF80,ZNF83,ZNF84,ZNF91,ZNHIT2,ZSCAN2,ZXDC,ZYX,ZZEF1,ZZZ3
22473,3.402084,3.494320,4.023385,3.541183,3.832442,6.952044,3.855832,4.082446,3.627827,4.335390,...,3.974718,5.558950,5.514646,6.841577,3.183982,4.412229,6.200891,6.647563,6.251839,5.097675
249608,4.187661,3.866810,4.085393,3.139782,4.328903,5.025544,3.661857,4.191745,3.685930,4.569808,...,3.842874,6.804398,7.363036,8.035668,4.077028,3.948575,6.492446,5.693120,6.294657,5.586593
249599,4.837456,4.006630,3.761547,3.688134,4.285157,6.048967,3.363964,3.470209,3.483006,4.521905,...,3.334548,7.619913,7.060920,8.144723,3.051310,3.543237,5.964417,6.366522,6.379608,6.199579
441791,3.187184,3.628224,3.539075,3.587210,3.619540,5.592908,3.491800,3.593392,3.615120,4.072719,...,3.500689,7.447879,5.975008,8.160910,3.569871,3.626728,6.146075,6.470966,5.285979,6.637736
249560,3.374551,3.497133,3.661740,3.460654,3.436120,5.450084,3.396684,3.840728,3.782517,4.735773,...,3.867339,5.562545,6.554106,7.087424,3.667209,3.522804,7.350276,6.389579,6.662883,4.938632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22502,3.791553,3.662776,4.079825,3.758397,3.395359,5.764780,3.540518,3.281390,3.479762,4.149927,...,3.464084,6.851393,6.865060,9.865863,3.543129,3.991906,6.949398,7.204528,5.047344,6.525453
441685,3.085443,3.653459,4.123113,3.680947,3.647998,5.939426,3.512556,3.546215,3.612820,4.027431,...,3.549222,5.925519,5.060669,6.356471,3.653427,3.766725,6.038282,5.789259,7.032159,6.530652
441899,3.187184,3.628224,3.539075,4.944182,3.619540,5.592908,3.491800,3.593392,3.615120,4.072719,...,3.500689,7.269916,7.222344,10.263361,3.569871,3.626728,6.107571,7.614689,6.006757,5.465672
305219,3.152498,3.654102,3.478021,3.613843,3.624577,7.054350,3.489415,3.558475,3.608941,4.076373,...,3.466420,6.817555,6.496405,9.184417,3.420080,3.521218,5.430783,7.427696,5.050724,6.400798


In [5]:
train_emb_abs_df = pd.read_csv("/var/www/datasets/embedding-vectors/tamoxifen_group/v2/absolute/property_vector_ge_TRAIN_2021-03-04.csv", sep="\t", index_col="patient_ID")
train_emb_abs_outcome_df = train_emb_abs_df.join(ge_out_df["posOutcome"])
X_train_emb_abs_df, y_train_emb_abs_df = train_emb_abs_outcome_df.drop(["posOutcome"], axis=1), train_emb_abs_outcome_df["posOutcome"]
X_train_emb_abs_df, y_train_emb_abs_df = X_train_emb_abs_df.loc[y_ge_train.index,:], y_train_emb_abs_df.loc[y_ge_train.index]
X_train_emb_abs_df

,0,1,2,3,4,5,6,7,8,9,...,438,439,440,441,442,443,444,445,446,447
22473,0.237823,-0.059521,0.230957,0.050294,0.133526,0.028123,-0.002614,0.066303,-0.125752,-0.008396,...,-0.000359,-0.002639,0.015120,0.003535,0.010068,0.003408,-0.001064,-0.000563,-0.003197,-2.881978e-06
249608,-0.173263,-0.187113,-0.001124,0.037108,-0.008250,-0.087975,-0.006915,-0.115483,0.048986,0.285757,...,0.009936,0.002214,-0.003510,0.002198,0.001158,-0.000322,-0.002706,0.003647,-0.000714,1.604505e-05
249599,0.201768,-0.029456,0.182885,-0.049242,-0.279091,0.298254,0.033760,0.065236,-0.084633,0.000219,...,-0.002821,-0.000035,0.004231,-0.002739,-0.008983,-0.006409,0.002831,-0.001213,-0.000136,1.924525e-05
441791,-0.140808,0.109703,0.027210,-0.068730,-0.141512,0.023152,0.034641,-0.068911,-0.065362,-0.038813,...,0.002569,-0.003972,-0.001924,0.002100,-0.001682,-0.001704,0.000939,-0.000230,0.000522,-5.176709e-06
249560,0.069039,0.031378,0.168199,0.071899,-0.042612,0.176539,-0.052509,-0.108884,-0.069096,0.088603,...,0.003032,-0.011236,-0.005964,0.002140,0.006396,0.013673,0.004449,0.005885,-0.002776,1.919820e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22502,0.024670,-0.161788,-0.091536,-0.177791,0.074665,0.146178,0.034686,0.010591,-0.096995,-0.057871,...,0.006561,-0.006026,0.000400,0.004194,0.001375,0.004093,0.001981,-0.003848,0.002427,-1.241575e-05
441685,0.162798,-0.146114,0.247207,-0.086093,0.131794,-0.152520,0.106691,0.095469,0.072779,0.005359,...,-0.012960,-0.002213,-0.002588,0.013510,-0.006051,0.004944,0.005721,0.002523,-0.000135,1.531776e-05
441899,0.083137,0.023575,0.193433,-0.057129,-0.029097,-0.055911,-0.057442,-0.070603,-0.167829,-0.124643,...,-0.000217,-0.005835,0.001702,0.002454,-0.005235,-0.016558,-0.000003,-0.000213,-0.001711,4.043089e-05
305219,-0.144892,0.275208,-0.052561,0.044638,-0.142087,0.049335,0.185052,-0.116351,-0.229087,0.056807,...,0.002822,-0.001944,0.002204,0.002188,-0.004815,-0.003226,0.001900,-0.000100,-0.000391,-2.115106e-07


In [7]:
test_emb_abs_df = pd.read_csv("/var/www/datasets/embedding-vectors/tamoxifen_group/v2/absolute/property_vector_ge_TEST_2021-03-04.csv", sep="\t", index_col="patient_ID")
test_emb_abs_outcome_df = test_emb_abs_df.join(ge_out_df["posOutcome"])
X_test_emb_abs_df, y_test_emb_abs_df = test_emb_abs_outcome_df.drop(["posOutcome"], axis=1), test_emb_abs_outcome_df["posOutcome"]
X_test_emb_abs_df, y_test_emb_abs_df = X_test_emb_abs_df.loc[y_ge_test.index,:], y_test_emb_abs_df[y_ge_test.index]

In [ ]:
X_train_emb_abs_34_dim, X_test_emb_abs_34_dim = X_train_emb_abs_df.iloc[:, :34], X_test_emb_abs_df.iloc[:, :34]

In [9]:
feats_100 = load_features("/var/www/datasets/feats_100_raw_nn.txt")

In [10]:
params_raw_acc, clf_raw_acc, cv_score_raw_acc, test_scores_raw_acc = evaluate_ge((X_ge_train, X_ge_test, y_ge_train, y_ge_test), split=False, feats=feats_100, rand_scoring="balanced_accuracy")

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[12:10:49] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:10:49] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Time taken: 0 hours 0 minutes and 3.99 seconds.
Best Score: 74.529%
{'subsample': 0.6, 'scale_pos_weight': 0.3, 'n_estimators': 50, 'min_child_weight': 1, 'max_depth': 5, 'max_delta_step': 4, 'learning_rate': 0.07, 'gamma': 1.5, 'colsample_bytree': 0.6}
[12:10:57] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric use

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:    3.8s finished


In [43]:
params_emb_acc, clf_emb_acc, cv_score_emb_acc, test_scores_emb_acc = evaluate_ge((X_train_emb_abs_df, X_test_emb_abs_df, y_ge_train, y_ge_test), split=False, rand_scoring="balanced_accuracy")

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[15:42:41] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:42:41] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Time taken: 0 hours 0 minutes and 26.75 seconds.
Best Score: 76.906%
{'subsample': 1.0, 'scale_pos_weight': 0.3, 'n_estimators': 20, 'min_child_weight': 4, 'max_depth': 6, 'max_delta_step': 4, 'learning_rate': 0.05, 'gamma': 1.5, 'colsample_bytree': 0.6}
[15:42:45] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric us

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:   26.2s finished


In [11]:
params_emb_34_acc, clf_emb_34_acc, cv_score_emb_34_acc, test_scores_emb_34_acc = evaluate_ge((X_train_emb_abs_34_dim, X_test_emb_abs_34_dim, y_ge_train, y_ge_test), split=False, rand_scoring="balanced_accuracy")

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[12:12:37] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:12:37] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Time taken: 0 hours 0 minutes and 1.62 seconds.
Best Score: 78.057%
{'subsample': 0.6, 'scale_pos_weight': 0.1, 'n_estimators': 120, 'min_child_weight': 1, 'max_depth': 5, 'max_delta_step': 3, 'learning_rate': 0.07, 'gamma': 1, 'colsample_bytree': 0.8}
[12:12:54] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  94 out of 125 | elapsed:    1.1s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:    1.5s finished


In [22]:
#Compare raw model with embedding model
print_score_comparison_v2(cv_score_raw_acc, cv_score_emb_34_acc, test_scores_raw_acc, test_scores_emb_34_acc, header_1="Raw", header_2="Emb (34 dims)")

	Validation set - Balanced Opt

			Raw				Emb (34 dims)
		-------------------------------------------------------
balanced_accuracy:	74.53%				78.06%

recall_0:		63.33%				88.33%

precision_0:		62.14%				50.24%

recall_1:		85.72%				67.78%

precision_1:		86.62%				94.31%

auc:			84.56%				85.71%



	Test set - Balanced Opt

			Raw				Emb (34 dims)
		-------------------------------------------------------
balanced_accuracy:	64.37%				66.48%

recall_0:		47.06%				74.51%

precision_0:		48.00%				39.18%

recall_1:		81.69%				58.45%

precision_1:		81.12%				86.46%

auc:			72.73%				71.06%



In [25]:
emb_cols = X_train_emb_abs_34_dim.columns.to_list()
X_ge_100_train, X_ge_100_test = X_ge_train[feats_100], X_ge_test[feats_100]

X_raw_emb_train = pd.merge(X_ge_100_train, X_train_emb_abs_34_dim, left_index=True, right_index=True)
X_raw_emb_test = pd.merge(X_ge_100_test, X_test_emb_abs_34_dim, left_index=True, right_index=True)
X_raw_emb_test

,MYH7B,ADRA2A,GRB7,LRP12,MSRA,MRPL17,HSPB3,EDC3,CACNA1H,GLDC,...,24,25,26,27,28,29,30,31,32,33
249540,4.081220,5.513807,4.859041,5.341824,4.396074,9.662919,2.676551,4.301549,6.221798,1.367038,...,0.074042,-0.087038,-0.024521,0.020580,-0.072276,-0.007125,-0.035043,-0.009919,-0.014100,0.041364
441885,3.346856,4.321042,4.915306,3.316025,5.162939,6.609321,2.824619,4.018516,3.205853,2.990778,...,0.054780,-0.014115,-0.034533,-0.010267,0.011849,-0.051400,-0.041256,-0.025779,-0.009573,0.016152
441672,3.344460,4.884633,9.315200,3.228507,4.330381,9.679237,2.803146,4.717642,3.252228,2.992718,...,0.048165,-0.064514,0.199639,0.125585,0.004960,0.118577,-0.068890,-0.089174,0.031909,-0.056347
441746,3.346856,5.651840,4.915306,3.316025,4.950791,8.296965,2.824619,4.180924,3.205853,2.990778,...,0.027432,-0.070788,-0.019618,-0.095424,-0.016514,-0.026200,-0.008108,0.077148,0.039271,0.066759
305151,3.346433,6.204796,4.894265,3.231599,5.866692,8.514736,2.873333,4.054779,3.192964,2.993512,...,0.007469,0.030973,-0.141391,0.014249,0.004854,-0.127573,-0.009636,-0.044794,0.007050,0.070754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441908,3.346856,4.300481,4.226472,3.316025,6.942172,6.957008,10.023862,4.122144,3.205853,3.008633,...,0.082466,0.110974,-0.013309,0.005476,0.104104,0.035150,0.013193,-0.026528,0.036514,0.027907
441675,3.344460,4.513820,4.422115,5.289645,6.456045,8.784964,2.803146,5.828296,3.252228,2.992718,...,-0.045137,-0.053588,0.000039,0.005054,-0.013336,-0.004936,-0.023907,0.034399,0.074089,-0.091005
441765,3.346856,5.901843,4.917949,3.316025,5.491019,7.469235,2.824619,4.018516,3.205853,2.990778,...,0.005693,-0.001616,0.034910,-0.020828,-0.053719,-0.005217,0.009449,-0.003725,0.014639,-0.049633
305133,3.346433,4.590048,5.432556,3.231599,5.078104,8.873241,2.873333,4.025407,6.156487,3.199910,...,0.142164,-0.008983,0.042038,-0.006486,0.002535,0.012338,0.039600,-0.003016,-0.003724,-0.077369


In [28]:
from mlxtend.classifier import StackingClassifier
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

raw_pipe = make_pipeline(ColumnSelector(cols=feats_100), clf_raw_acc)
emb_pip = make_pipeline(ColumnSelector(cols=emb_cols), clf_emb_34_acc)
sclf_acc_v1 = StackingClassifier(classifiers=[raw_pipe, emb_pip], meta_classifier=LogisticRegression())

In [33]:
cv_results_sclf_1 = cross_validate(sclf_acc_v1, X_raw_emb_train, y_train_emb_abs_df,
                                   n_jobs=-1, scoring=scoring, cv=st_cv)

cv_results_sclf_1_df = get_scores(cv_results_sclf_1)
cv_results_sclf_1_df.mean()


balanced_accuracy      0.748700
recall_0               0.625000
precision_0            0.642282
recall_1               0.872401
precision_1            0.865407
auc                    0.824114
specificity            0.625000
average_precision_0    0.219227
dtype: float64

In [34]:
sclf_acc_v1.fit(X_raw_emb_train, y_ge_train)
test_scores_ens = calc_scores(sclf_acc_v1, X_raw_emb_test, y_ge_test)
test_scores_ens.mean()

[12:40:35] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:40:37] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


balanced_accuracy      0.631939
recall_0               0.411765
precision_0            0.500000
recall_1               0.852113
precision_1            0.801325
auc                    0.711958
specificity            0.411765
average_precision_0    0.225676
dtype: float64

In [36]:
#Compare stacked ensemble with raw base model
print_score_comparison_v2(cv_score_raw_acc, cv_results_sclf_1_df, test_scores_raw_acc, test_scores_ens, header_1="Raw", header_2="Ensemble")

	Validation set - Balanced Opt

			Raw				Ensemble
		-------------------------------------------------------
balanced_accuracy:	74.53%				74.87%

recall_0:		63.33%				62.50%

precision_0:		62.14%				64.23%

recall_1:		85.72%				87.24%

precision_1:		86.62%				86.54%

auc:			84.56%				82.41%



	Test set - Balanced Opt

			Raw				Ensemble
		-------------------------------------------------------
balanced_accuracy:	64.37%				63.19%

recall_0:		47.06%				41.18%

precision_0:		48.00%				50.00%

recall_1:		81.69%				85.21%

precision_1:		81.12%				80.13%

auc:			72.73%				71.20%



In [37]:
#Compare stacked ensemble with embedding base model
print_score_comparison_v2(cv_score_emb_34_acc, cv_results_sclf_1_df, test_scores_emb_34_acc, test_scores_ens, header_1="Emb(34 dim)", header_2="Ensemble")

	Validation set - Balanced Opt

			Emb(34 dim)				Ensemble
		-------------------------------------------------------
balanced_accuracy:	78.06%				74.87%

recall_0:		88.33%				62.50%

precision_0:		50.24%				64.23%

recall_1:		67.78%				87.24%

precision_1:		94.31%				86.54%

auc:			85.71%				82.41%



	Test set - Balanced Opt

			Emb(34 dim)				Ensemble
		-------------------------------------------------------
balanced_accuracy:	66.48%				63.19%

recall_0:		74.51%				41.18%

precision_0:		39.18%				50.00%

recall_1:		58.45%				85.21%

precision_1:		86.46%				80.13%

auc:			71.06%				71.20%



In [38]:
params_raw_emb_acc, clf_raw_emb_acc, cv_score_raw_emb_acc, test_scores_raw_emb_acc = evaluate_ge((X_raw_emb_train, X_raw_emb_test, y_ge_train, y_ge_test), split=False, rand_scoring="balanced_accuracy")

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[12:43:54] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:43:54] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Time taken: 0 hours 0 minutes and 4.51 seconds.
Best Score: 79.293%
{'subsample': 1.0, 'scale_pos_weight': 0.3, 'n_estimators': 50, 'min_child_weight': 5, 'max_depth': 3, 'max_delta_step': 4, 'learning_rate': 0.05, 'gamma': 1, 'colsample_bytree': 0.6}
[12:44:00] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:    4.4s finished


In [39]:
#Compare raw with raw+emb model
print_score_comparison_v2(cv_score_raw_acc, cv_score_raw_emb_acc, test_scores_raw_acc, test_scores_raw_emb_acc, header_1="Raw", header_2="Raw + Emb")

	Validation set - Balanced Opt

			Raw				Raw + Emb
		-------------------------------------------------------
balanced_accuracy:	74.53%				79.29%

recall_0:		63.33%				78.33%

precision_0:		62.14%				59.23%

recall_1:		85.72%				80.25%

precision_1:		86.62%				91.04%

auc:			84.56%				85.48%



	Test set - Balanced Opt

			Raw				Raw + Emb
		-------------------------------------------------------
balanced_accuracy:	64.37%				65.83%

recall_0:		47.06%				54.90%

precision_0:		48.00%				45.90%

recall_1:		81.69%				76.76%

precision_1:		81.12%				82.58%

auc:			72.73%				72.98%



In [40]:
#Compare emb with raw + emb model
print_score_comparison_v2(cv_score_emb_34_acc, cv_score_raw_emb_acc, test_scores_emb_34_acc, test_scores_raw_emb_acc, header_1="Emb(34 dim)", header_2="Raw + Emb")

	Validation set - Balanced Opt

			Emb(34 dim)				Raw + Emb
		-------------------------------------------------------
balanced_accuracy:	78.06%				79.29%

recall_0:		88.33%				78.33%

precision_0:		50.24%				59.23%

recall_1:		67.78%				80.25%

precision_1:		94.31%				91.04%

auc:			85.71%				85.48%



	Test set - Balanced Opt

			Emb(34 dim)				Raw + Emb
		-------------------------------------------------------
balanced_accuracy:	66.48%				65.83%

recall_0:		74.51%				54.90%

precision_0:		39.18%				45.90%

recall_1:		58.45%				76.76%

precision_1:		86.46%				82.58%

auc:			71.06%				72.98%

